###Spark Join exercise

####INNER JOIN

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
movie_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
            .filter(f"file_date = '{v_file_date}'")

In [0]:
language_df = spark.read.format("delta").load(f"{silver_folder_path}/language")

In [0]:
genre_df = spark.read.format("delta").load(f"{silver_folder_path}/genre")

In [0]:
#movie_genre_df = spark.read.parquet(f"{silver_folder_path}/movie_genre") \
#            .filter(f"file_date = '{v_file_date}'")
movie_genre_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_genre") \
          .filter(f"file_date = '{v_file_date}'") # query table by path            

In [0]:
movie_language_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_languages")\
            .filter(f"file_date = '{v_file_date}'")

In [0]:
movie_df.limit(10).display()

In [0]:
movie_genre_df.limit(10).display()

In [0]:
language_df.limit(10).display()

In [0]:
movie_language_df.limit(10).display()

In [0]:
genre_df.limit(10).display()

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movies_languages_df = language_df.join(movie_language_df,
                                       "language_id",
                                       "inner") \
                                 .select(language_df.language_name, language_df.language_id, movie_language_df.movie_id)

In [0]:
movies_genres_df = genre_df.join(movie_genre_df,
                                 "genre_id",
                                 "inner") \
                                 .select(genre_df.genre_name, genre_df.genre_id, movie_genre_df.movie_id)

In [0]:
movie_filter_df = movie_df.filter("year_release_date >= 2000")

In [0]:
results_movie_genres_languages_df = movie_filter_df \
                                .join(movies_languages_df,
                                      "movie_id",
                                      "inner") \
                                .join(movies_genres_df,
                                      "movie_id",
                                      "inner") \
                                .select(movie_filter_df.movie_id,movies_languages_df.language_id,movies_genres_df.genre_id, movie_df.title, movie_df.duration_time, movie_df.release_date, movie_df.vote_average,movies_languages_df.language_name,movies_genres_df.genre_name) \
                                .withColumn("created_date", lit(v_file_date)) \
                                .orderBy("release_date", descending=True)

display(results_movie_genres_languages_df)

In [0]:
#overwrite_partition(results_movie_genres_languages_df,"movie_gold","results_movie_genres_languages","created_date")

In [0]:
#results_movie_genres_languages_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_movie_genres_languages")
merge_condition='tgt.movie_id = src.movie_id and tgt.language_id = src.language_id and tgt.genre_id = src.genre_id and tgt.created_date = src.created_date'
# Remove duplicates based on merge keys before merge
deduped_df = results_movie_genres_languages_df.dropDuplicates([
    "movie_id",
    "language_id",
    "genre_id",
    "created_date"
])
merge_delta_lake(
    deduped_df,
    "movie_gold",
    "results_movie_genres_languages",
    gold_folder_path,
    merge_condition,
    "created_date"
)
#merge_delta_lake(results_movie_genres_languages_df, "movie_gold", "results_movie_genres_languages", gold_folder_path, merge_condition, "created_date")

#results_movie_genres_languages_df.write.mode("append").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_movie_genres_languages")

In [0]:
#results_movie_genres_languages_df.write.mode("append").partitionBy("created_date").format("delta").save(f"{gold_folder_path}/results_movie_genres_languages")

In [0]:
%sql
select * from movie_gold.results_movie_genres_languages

In [0]:
display(spark.read.format('delta').load(f"{gold_folder_path}/results_movie_genres_languages"))